# SETUP

In [1]:
!git clone https://github.com/cloneofsimo/lora.git && pip install /content/lora
!pip install accelerate bitsandbytes

Cloning into 'lora'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 221 (delta 64), reused 67 (delta 46), pack-reused 125
Receiving objects: 100% (221/221), 47.62 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./lora
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 503 kB 36.5 MB/s 
     |████████████████████████████████| 5.8 MB 60.9 MB/s 
     |████████████████████████████████|

# TRAINING

In [ ]:
import os
import shutil
from google.colab import files
from tqdm import tqdm

PRETRAINED_MODEL="runwayml/stable-diffusion-v1-5" #@param{type: 'string'}
PROMPT="dhoniqaz" #@param{type: 'string'}

OUTPUT_DIR="" #@param{type: 'string'}
IMAGES_FOLDER_OPTIONAL="" #@param{type: 'string'}

RESOLUTION="512" #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"]
RESOLUTION=int(RESOLUTION)

if PRETRAINED_MODEL == "":
  print('[1;31mYou should define the pretrained model.')

else:
  if IMAGES_FOLDER_OPTIONAL=="":
    INSTANCE_DIR = "/content/data_example"
    if not os.path.exists(str(INSTANCE_DIR)):
      %mkdir -p "$INSTANCE_DIR"
    uploaded = files.upload()
    for filename in tqdm(uploaded.keys(), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
        shutil.move(filename, INSTANCE_DIR)
  else:
    INSTANCE_DIR = IMAGES_FOLDER_OPTIONAL
  
  if OUTPUT_DIR == "":
    OUTPUT_DIR = "/content/output"
  if not os.path.exists(str(OUTPUT_DIR)):
    %mkdir -p "$OUTPUT_DIR"

In [ ]:
STEPS = 100 #@param {type:"slider", min:0, max:10000, step:10}
BATCH_SIZE = 5 #@param {type:"slider", min:0, max:128, step:1}
FP_16 = True #@param {type:"boolean"}

#@markdown ----
#@markdown UNET PARAMS
LEARNING_RATE = 1e-4 #@param {type:"number"}

#@markdown ----
TRAIN_TEXT_ENCODER = True #@param {type:"boolean"}
#@markdown TEXT ENCODER PARAMS
LEARNING_RATE_TEXT_ENCODER = 5e-5 #@param {type:"number"}

NEW_LEARNING_RATE = LEARNING_RATE / BATCH_SIZE
NEW_LEARNING_RATE_TEXT_ENCODER = LEARNING_RATE_TEXT_ENCODER / BATCH_SIZE

if FP_16:
  fp_16_arg = "fp16"
else:
  fp_16_arg = ""

if TRAIN_TEXT_ENCODER:
  !accelerate launch lora/train_lora_dreambooth.py \
    --pretrained_model_name_or_path="$PRETRAINED_MODEL" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --instance_prompt="$PROMPT" \
    --resolution=512 \
    --use_8bit_adam \
    --mixed_precision="$fp_16_arg" \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --learning_rate=$NEW_LEARNING_RATE \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=$STEPS \
    --train_text_encoder \
    --learning_rate_text=$NEW_LEARNING_RATE_TEXT_ENCODER
else:
  !accelerate launch lora/train_lora_dreambooth.py \
    --pretrained_model_name_or_path="$PRETRAINED_MODEL" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --instance_prompt="$PROMPT" \
    --resolution=512 \
    --use_8bit_adam \
    --mixed_precision="$fp_16_arg" \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --learning_rate=$NEW_LEARNING_RATE \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=$STEPS

# INFERENCE

In [ ]:
#@title LOADING MODEL AND MONKEY PATCHING IT
import torch
from lora_diffusion import monkeypatch_lora, tune_lora_scale
from diffusers import StableDiffusionPipeline


pipe = StableDiffusionPipeline.from_pretrained(PRETRAINED_MODEL, torch_dtype=torch.float16).to("cuda")
monkeypatch_lora(pipe.unet, torch.load(os.path.join(OUTPUT_DIR, "lora_weight.pt")))
monkeypatch_lora(pipe.text_encoder, torch.load(os.path.join(OUTPUT_DIR, "lora_weight.text_encoder.pt")), target_replace_module=["CLIPAttention"])

In [ ]:
INFERENCE_PROMPT = 'photo of dhoniqaz , synthwave' #@param {type:"string"}
LORA_SCALE_UNET = 1 #@param {type:"number"}
LORA_SCALE_TEXT_ENCODER = 1 #@param {type:"number"}
GUIDANCE = 7 #@param {type:"slider", min:0, max:15, step:0.2}
tune_lora_scale(pipe.unet, LORA_SCALE_UNET)
if TRAIN_TEXT_ENCODER:
  tune_lora_scale(pipe.text_encoder, LORA_SCALE_TEXT_ENCODER)
image = pipe(INFERENCE_PROMPT, num_inference_steps=50, guidance_scale=GUIDANCE).images[0]
image